In [ ]:
import os
import glob
from trainval_net import train

In [ ]:
# write files in DCMImagesTest to text file in ImageSets
base_path = "/opt/R-FCN.pytorch/data/PNAdevkit/PNA2018"
d = os.path.join(base_path, 'DCMImagesTrain')
pids = [pid.split('.')[0] for pid in os.listdir(d)]

ImageSets_dir = os.path.join(base_path, 'ImageSets')
if not os.path.exists(ImageSets_dir):
    os.mkdir(ImageSets_dir)

with open(os.path.join(ImageSets_dir,'train.txt'), 'w') as f:
    for pid in pids:
        f.write("{}\n".format(pid))

# Training

We trained our final submission based on 4 models, while keeping all the other settings exactly the same, only the `RNG_SEED` was changed. NOTE: `session` is just a variable for naming output files. The image files were

The below code will generate 4 models in the `weights` directory in the same directory as where `run.sh` command was invoked. NOTE: if a override directory for `weights` was set by using the option `--weights` when invoking `run.sh`, the weights will be saved accordingly.

NOTE: Change the `num_workers` to the number of GPUs specified when invoking `run.sh` script. Otherwise, it will be set to default, 2 GPUS with ids 0,1

In [ ]:
def delete_older_checkpoints(path):
    files = list(filter(os.path.isfile, glob.glob(path)))
    files.sort(key=lambda x: os.path.getmtime(x), reverse=False)
    files.pop()
    for f in files:
        print("removing older checkpoint file {}".format(f))
        os.remove(f)

In [ ]:
for i, random_seed in enumerate([14, 18, 20, 30]):
    print("\n\n********* STARTING TRAINING FOR RANDOM SEED {} ({}/4) *********\n\n".format(random_seed, i))
    train_config = {"session": i, "max_epochs": 14, "num_workers": 2, "optimizer": "sgd", "batch_size": 4, 
                    "flip_prob": 0.5, "RNG_SEED": random_seed, 
                    "TRAIN": {"LEARNING_RATE": 0.001, "FG_THRESH": 0.3, "BG_THRESH_HI": 0.3, "BG_THRESH_LO": 0.0}}
    train(**train_config)
    delete_older_checkpoints("/notebooks/save/couplenet/res152/kaggle_pna/couplenet_{}_*.pth".format(i))